In [8]:
import sys
import transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from collections import defaultdict
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [12]:
import pandas as pd
import numpy as np
import math
from statistics import mean
import pickle
from scipy import sparse

In [13]:
# double check the path on harddrive
import pathlib
print(pathlib.Path().resolve())

/Users/kennethshinn/Documents/fyoudie/python_workbooks


In [14]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
class SentimentClassifier(nn.Module):

  def __init__(self):
    super(SentimentClassifier, self).__init__()
    self.bert = BertModel.from_pretrained('bert-base-cased')
    self.drop = nn.Dropout(p=0.3)
    self.out = nn.Linear(self.bert.config.hidden_size, 3)
  
  def forward(self, input_ids, attention_mask):
    bert_out = self.bert(
      input_ids=input_ids,
      attention_mask=attention_mask
    )
    pooled_output = bert_out['pooler_output']
    # print(pooled_output)
    output = self.drop(pooled_output)
    prob = F.softmax(self.out(output))
    return prob

In [16]:
MAX_LEN = 250

In [17]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [20]:
# model = torch.load('../models/bert_sentiment_model.pt', map_location=torch.device('cpu'))
model = SentimentClassifier()
model.load_state_dict(torch.load('../models/bert_sentiment_model.pt', map_location=torch.device('cuda')))
model = model.to(device)

ImportError: 
BertModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.


In [21]:
def get_sentence_sentiment(s):
    text = s
    encoded_review = tokenizer.encode_plus(
    text,
    max_length=MAX_LEN,
    add_special_tokens=True,
    return_token_type_ids=False,
    pad_to_max_length=True,
    return_attention_mask=True,
    return_tensors='pt',
    )
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)

    output = model(input_ids, attention_mask)
    return int(torch.argmax(output) - 1)


In [22]:
with open("../pickles/very_big_masks.pkl","rb") as f:
    mask_dict = pickle.load(f)

# # need to process this to be able to use it
# mask_vectors = list(mask_dict.values())
mask_vectors = [i.todense() for i in mask_dict]
mask_matrix = np.stack(mask_vectors)

with open("../pickles/very_big_restaurant_tfidf_dict.pkl","rb") as f:
    rid_sid_dict = pickle.load(f)

with open("../pickles/very_big_restaurant_sentence_list_dict.pkl","rb") as f:
    rid_tfidf_dict = pickle.load(f)

In [23]:
# iterate through the rids
rids = list(rid_sid_dict.keys())
num_rids = len(rids)

print(num_rids)

50763


In [11]:
# da meat

rid_feature_dict = {}
rid_agg_vec_dict = {}

cum_times = [time.perf_counter()]
op_times = []

for i in range(num_rids):
    # every 1000 rids, save the dicts
    if i % 1000 == 0 and i > 0:
        with open('../pickles/backups/rid_feature_dict_incomplete_' + str(i) + '.pkl', 'wb') as handle:
            pickle.dump(rid_feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open('../pickles/backups/rid_agg_vec_dict_incomplete_' + str(i) + '.pkl', 'wb') as handle:
            pickle.dump(rid_agg_vec_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    rid = rids[i]
    # get the sentences in order
    sentences = list(rid_sid_dict[rid].values)
    sentiments = np.array([get_sentence_sentiment(s) for s in sentences])
     
    tfidf_matrix = rid_tfidf_dict[rid]
    tfidf_matrix_w = tfidf_matrix.copy()
    tfidf_matrix_w = (tfidf_matrix_w.T.multiply(sentiments)).T # multiply the sentiments through for weighted matrix
        
    # sum over all columns of the matrix, for both weighted and not weighted
    freq_vec = tfidf_matrix.sum(axis=0).A1
    freq_vec_w = tfidf_matrix_w.sum(axis=0).A1
    
    # normalize the vectors
    freq_vec = freq_vec / np.linalg.norm(freq_vec)
    freq_vec_w = freq_vec_w / np.linalg.norm(freq_vec_w)
    
    # save the weighted and non weighted vectors (for future mask updates), keep it sparse for memory constraints
    rid_agg_vec_dict[rid] = (sparse.csr_matrix(freq_vec), sparse.csr_matrix(freq_vec_w))
    
    # apply the mask for params and save both weighted and non weighted vectors for this rid
    rid_feature_dict[rid] = (np.matmul(mask_matrix, freq_vec.T).T, np.matmul(mask_matrix, freq_vec_w.T).T)
    
    # update the timer and track progress
    curr_time = time.perf_counter()
    op_time = curr_time - cum_times[-1]
    cum_times.append(curr_time)
    op_times.append(op_time)
    if i % 10 == 0 and i > 0 :
        print('mean op time', mean(op_times), ' seconds')
        print((i / num_rids) * 100, '% complete')
        print('time left: ', mean(op_times)  * (num_rids - i), ' seconds')
        print()

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


mean op time 52.01187457272727  seconds
0.019699387349053445 % complete
time left:  2639758.6701896274  seconds

mean op time 48.587937371428566  seconds
0.03939877469810689 % complete
time left:  2465497.7060383996  seconds

mean op time 56.55124852580646  seconds
0.059098162047160334 % complete
time left:  2869014.491459739  seconds

mean op time 64.62310109268293  seconds
0.07879754939621378 % complete
time left:  3277877.5567241563  seconds

mean op time 58.555826884313724  seconds
0.09849693674526723 % complete
time left:  2969541.648784202  seconds

mean op time 55.509096773770494  seconds
0.11819632409432067 % complete
time left:  2814477.7337204856  seconds

mean op time 53.10531341549296  seconds
0.1378957114433741 % complete
time left:  2692067.6529715843  seconds

mean op time 49.7572189037037  seconds
0.15759509879242756 % complete
time left:  2521845.1256964146  seconds

mean op time 57.6118962923077  seconds
0.177294486141481 % complete
time left:  2919367.620820108  seco

mean op time 34.08845603475366  seconds
1.4774540511790084 % complete
time left:  1704865.9516661349  seconds

mean op time 33.79830651432326  seconds
1.4971534385280618 % complete
time left:  1690016.7206357059  seconds

mean op time 33.711628989753564  seconds
1.5168528258771152 % complete
time left:  1685345.46808475  seconds

mean op time 33.60387908540333  seconds
1.5365522132261686 % complete
time left:  1679622.6883257146  seconds

mean op time 33.4414795562579  seconds
1.5562516005752223 % complete
time left:  1671171.0578648762  seconds

mean op time 33.20914096142322  seconds
1.5759509879242757 % complete
time left:  1659228.3098555885  seconds

mean op time 33.1148898595561  seconds
1.5956503752733289 % complete
time left:  1654188.093154406  seconds

mean op time 32.936717945676  seconds
1.6153497626223825 % complete
time left:  1644958.5043608965  seconds

mean op time 32.71545471131167  seconds
1.635049149971436 % complete
time left:  1633580.8000999256  seconds

mean op 

mean op time 25.47478690898726  seconds
2.935208715008963 % complete
time left:  1255219.1753665293  seconds

mean op time 25.38462888434377  seconds
2.954908102358017 % complete
time left:  1250522.972729427  seconds

mean op time 25.280421732296492  seconds
2.97460748970707 % complete
time left:  1245136.611580799  seconds

mean op time 25.199517721827746  seconds
2.9943068770561236 % complete
time left:  1240899.8511759636  seconds

mean op time 25.0955303280209  seconds
3.014006264405177 % complete
time left:  1235528.244639453  seconds

mean op time 25.052637789487346  seconds
3.0337056517542305 % complete
time left:  1233165.9899119355  seconds

mean op time 25.006305028820115  seconds
3.053405039103284 % complete
time left:  1230635.2893833243  seconds

mean op time 24.901551471620756  seconds
3.0731044264523373 % complete
time left:  1225231.037058156  seconds

mean op time 24.81292783112667  seconds
3.0928038138013907 % complete
time left:  1220622.3587966145  seconds

mean op

mean op time 20.097468270281123  seconds
4.412662766187971 % complete
time left:  975189.452878851  seconds

mean op time 20.042381780764106  seconds
4.432362153537025 % complete
time left:  972316.0673302091  seconds

mean op time 19.992230359177356  seconds
4.452061540886079 % complete
time left:  969683.1491111793  seconds

mean op time 19.93121570686922  seconds
4.471760928235132 % complete
time left:  966524.4432732092  seconds

mean op time 19.8703688313459  seconds
4.4914603155841855 % complete
time left:  963375.0920501433  seconds

mean op time 19.815668408642512  seconds
4.511159702933239 % complete
time left:  960524.8947721286  seconds

mean op time 19.75970875697523  seconds
4.530859090282292 % complete
time left:  957614.7654892905  seconds

mean op time 19.700505243574213  seconds
4.550558477631346 % complete
time left:  954548.5805669014  seconds

mean op time 19.645765477423524  seconds
4.570257864980399 % complete
time left:  951699.8170228278  seconds

mean op time 1

...

mean op time 11.277884624491527  seconds
92.88261135078699 % complete
time left:  40746.99714828789  seconds

mean op time 11.277412404425268  seconds
92.90231073813604 % complete
time left:  40632.51689314424  seconds

mean op time 11.277969980640648  seconds
92.9220101254851 % complete
time left:  40521.74614044185  seconds

mean op time 11.279412685769696  seconds
92.94170951283415 % complete
time left:  40414.13565311282  seconds

mean op time 11.280687903905404  seconds
92.9614089001832 % complete
time left:  40305.89788065401  seconds

mean op time 11.280527409162941  seconds
92.98110828753225 % complete
time left:  40192.51915884756  seconds

mean op time 11.280372172599607  seconds
93.00080767488132 % complete
time left:  40079.16232924641  seconds

mean op time 11.279705440384575  seconds
93.02050706223037 % complete
time left:  39963.99637528255  seconds

mean op time 11.280814218113102  seconds
93.04020644957941 % complete
time left:  39855.11663259359  seconds

mean op time

mean op time 11.2610552052525  seconds
94.360065401966 % complete
time left:  32240.401052637906  seconds

mean op time 11.260893015704118  seconds
94.37976478931506 % complete
time left:  32127.32777380385  seconds

mean op time 11.25972786769266  seconds
94.39946417666411 % complete
time left:  32011.406327850233  seconds

mean op time 11.259981334207508  seconds
94.41916356401316 % complete
time left:  31899.52711980987  seconds

mean op time 11.259309007534261  seconds
94.43886295136221 % complete
time left:  31785.02932826922  seconds

mean op time 11.258016691305707  seconds
94.45856233871127 % complete
time left:  31668.800952642956  seconds

mean op time 11.257558343948208  seconds
94.47826172606032 % complete
time left:  31554.936038086827  seconds

mean op time 11.256584233355568  seconds
94.49796111340937 % complete
time left:  31439.6397637621  seconds

mean op time 11.255528894489485  seconds
94.51766050075842 % complete
time left:  31324.136913364237  seconds

mean op tim

mean op time 11.181467410959693  seconds
95.837519453145 % complete
time left:  23626.440639357832  seconds

mean op time 11.179629816413554  seconds
95.85721884049406 % complete
time left:  23510.761503917704  seconds

mean op time 11.178470555624498  seconds
95.8769182278431 % complete
time left:  23396.538872922076  seconds

mean op time 11.176995960276084  seconds
95.89661761519217 % complete
time left:  23281.682585255083  seconds

mean op time 11.175673514353782  seconds
95.91631700254122 % complete
time left:  23167.171195255392  seconds

mean op time 11.174392418233714  seconds
95.93601638989028 % complete
time left:  23052.77155881615  seconds

mean op time 11.172691158089547  seconds
95.95571577723932 % complete
time left:  22937.53494755784  seconds

mean op time 11.170849853777632  seconds
95.97541516458838 % complete
time left:  22822.046251267704  seconds

mean op time 11.16931862194291  seconds
95.99511455193743 % complete
time left:  22707.224758409935  seconds

mean op

mean op time 11.060352269533006  seconds
97.31497350432402 % complete
time left:  15075.260143373487  seconds

mean op time 11.058399775369857  seconds
97.33467289167307 % complete
time left:  14962.014896075416  seconds

mean op time 11.056718562738512  seconds
97.35437227902213 % complete
time left:  14849.173029757821  seconds

mean op time 11.055031346205823  seconds
97.37407166637118 % complete
time left:  14736.356784492362  seconds

mean op time 11.053065682967578  seconds
97.39377105372023 % complete
time left:  14623.205898566106  seconds

mean op time 11.051223054097996  seconds
97.41347044106928 % complete
time left:  14510.255870030669  seconds

mean op time 11.049447434154182  seconds
97.43316982841833 % complete
time left:  14397.430006702898  seconds

mean op time 11.0478739289887  seconds
97.4528692157674 % complete
time left:  14284.90099018239  seconds

mean op time 11.046012153755987  seconds
97.47256860311644 % complete
time left:  14172.03359326893  seconds

mean o

mean op time 10.919444167356582  seconds
98.79242755550302 % complete
time left:  6693.619274589585  seconds

mean op time 10.917439277703794  seconds
98.81212694285207 % complete
time left:  6583.215884455388  seconds

mean op time 10.915480356787787  seconds
98.83182633020114 % complete
time left:  6472.879851575158  seconds

mean op time 10.913528891512723  seconds
98.85152571755017 % complete
time left:  6362.587343751918  seconds

mean op time 10.911565604024625  seconds
98.87122510489924 % complete
time left:  6252.32709110611  seconds

mean op time 10.90971407673154  seconds
98.89092449224829 % complete
time left:  6142.169025199857  seconds

mean op time 10.907792360791461  seconds
98.91062387959735 % complete
time left:  6032.009175517678  seconds

mean op time 10.905977257097629  seconds
98.9303232669464 % complete
time left:  5921.945650604012  seconds

mean op time 10.904067054709245  seconds
98.95002265429545 % complete
time left:  5811.867740160027  seconds

mean op time 

In [12]:
# pickle the features
# with open('../pickles/rid_feature_dict.pkl', 'wb') as handle:
#     pickle.dump(rid_feature_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('../pickles/rid_agg_vec_dict.pkl', 'wb') as handle:
#     pickle.dump(rid_agg_vec_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
with open('../pickles/rid_feature_dict.pkl',"rb") as f:
    rid_feature_dict = pickle.load(f)

In [25]:
# we have to post process the vectors so that we extract a parameter term that is weighted or unweighted
# price = weighted
# taste = weighted 
# noise = unweighted
# decor = weighted
# saltiness = unweighted
# msg = unweighted 
# cleaniless = weighted
# oiliness = unweighted

weighted = np.matrix([1,1,0,1,0,0,1,0])
unweighted = np.matrix([1,1,1,1,1,1,1,1]) - weighted

rid_feature_dict_combined = {}
for key, value in rid_feature_dict.items():
    rid_feature_dict_combined[key] = np.multiply(unweighted, value[0].T) +  np.multiply(weighted, value[1].T)

In [34]:
# matching these with the people vectors, test
from numpy import genfromtxt
user_features = np.matrix(genfromtxt("../user_restaurant_data/user_features_test.csv", delimiter=','))


# Order is price, taste, noise, decor, saltiness, msg, cleanliness, and oiliness
# for every person in this file, dot the person with the feature vectors, and sort
user_sorted_rid = {}
cnt = 0
for row in user_features:
    print(row)
    user_feature_vector = row
    rid_scores = []
    for key, value in rid_feature_dict_combined.items():
        rid_scores.append((key, float(np.dot(user_feature_vector,value.T))))
    user_sorted_rid[cnt] = sorted(rid_scores, key=lambda x: x[1], reverse = True)
    cnt += 1

[[ 4.e+00  0.e+00  0.e+00  5.e+00  0.e+00  0.e+00  0.e+00 -1.e+05]]
[[1.  1.  0.  0.5 0.  0.  0.  0. ]]


In [35]:
rec_list = [x for (x,y) in user_sorted_rid[0]][:100]

In [37]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [38]:
import json

# import restaurant data from files

business_file_path = "../yelp_dataset/yelp_academic_dataset_business.json"

business_info_dict = {}
with open(business_file_path, 'rb') as business_data:
    for b in business_data:
        b_dict = json.loads(b)

        bid = b_dict['business_id']
        category = b_dict["categories"]
        if (category is not None and "Restaurants" in category):
            business_info_dict[bid] = b_dict

business_info_df = pd.DataFrame(business_info_dict.values())

In [39]:
rid_dict = pd.read_pickle("../script_data/new_to_old_restaurant_ids.pkl")

In [40]:
rid_dict = {v:k for k,v in rid_dict.items()}

In [41]:
int_id_list = rec_list
old_id_list = [rid_dict[int_id] for int_id in int_id_list]

In [42]:
business_info_df.query("business_id in {}".format(old_id_list))

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
1052,9ZW3MfYR9EInWWiJz3J0xQ,Table 153,228 Lonsdale Avenue,North Vancouver,BC,V7M 2G1,49.313011,-123.077230,4.0,58,0,"{'RestaurantsDelivery': 'False', 'WiFi': ''fre...","Restaurants, Cafes, Breakfast & Brunch","{'Tuesday': '8:0-16:0', 'Wednesday': '8:0-16:0..."
1148,b5eA9mluSKUngvkJ4ZCILA,Chimi King Buffet,5731 S Orange Blossom Trl,Orlando,FL,32839,28.475253,-81.396198,4.5,6,0,"{'Alcohol': 'u'full_bar'', 'RestaurantsReserva...","Restaurants, Caribbean, Latin American, Buffet...","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'..."
1442,5aIfAZCVQQ2b39eNytPHgw,Caffè Nero,560 Washington St,Boston,MA,02111,42.353511,-71.062326,4.0,443,1,"{'WiFi': ''free'', 'RestaurantsPriceRange2': '...","Coffee & Tea, Restaurants, Food, Breakfast & B...","{'Monday': '6:0-21:30', 'Tuesday': '6:0-21:30'..."
1501,eb4NLj2AM8s1JhOx9kn1KQ,The Ridge Golf and Gardens,17487 Robinson Rd,Marysville,OH,43040,40.178089,-83.346096,4.0,5,1,"{'GoodForKids': 'False', 'RestaurantsTableServ...","Sports Bars, Golf, Active Life, Venues & Event...","{'Monday': '0:0-0:0', 'Tuesday': '7:0-21:0', '..."
1872,viUq9OyTgOGgcdyuNoey3w,The Garden Strathcona,868 E Hastings Street,Vancouver,BC,V6A 1R6,49.280807,-123.085638,5.0,27,1,"{'GoodForMeal': '{'dessert': False, 'latenight...","Food, Flowers & Gifts, Shopping, Bakeries, Caf...","{'Monday': '0:0-0:0', 'Tuesday': '10:0-15:0', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48094,zO080AWINvqyJEsPzkchew,Colabbo,"1869 Cobb Pkwy SE, Ste 100",Marietta,GA,30060,33.907342,-84.488992,3.5,5,1,"{'RestaurantsDelivery': 'True', 'Music': '{'dj...","Restaurants, Beer Bar, Nightlife, Bars, Seafoo...","{'Monday': '15:0-22:0', 'Tuesday': '15:0-22:0'..."
49277,hrlUWrnN8ORaE9OTn-8TKg,Lear Faye,2270 Commercial Drive,Vancouver,BC,V5N,49.264421,-123.069712,4.5,21,0,"{'WiFi': 'u'no'', 'RestaurantsDelivery': 'Fals...","Cafes, Local Flavor, Food, Canadian (New), Cof...","{'Monday': '9:0-19:0', 'Tuesday': '9:0-19:0', ..."
49768,JcSs75E0yL_JtDOEyf5K8g,Cho Sun BBQ Korean Restaurant,788 Jervis Street,Vancouver,BC,V6E 0B5,49.287201,-123.127869,3.5,6,0,"{'RestaurantsDelivery': 'False', 'GoodForKids'...","Korean, Restaurants",None
50510,3L7p3J2RT7CDnxfNGnPZrg,Che-licious Desserts & Snacks,1120 Denman Street,Vancouver,BC,V6G 2M8,49.288094,-123.140537,4.5,45,0,"{'Caters': 'True', 'BusinessParking': '{'garag...","Food, Cafes, Restaurants, Desserts, Sandwiches...","{'Monday': '11:30-22:0', 'Tuesday': '11:30-22:..."
